In [3]:
import sqlite3
!pip install fastapi
!pip install uvicorn pydantic
from fastapi import FastAPI, Query
from pydantic import BaseModel
import uvicorn
import requests

app = FastAPI()

DB_FILE = "organization.db"

def initialize_db():
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS Employees (
            ID INTEGER PRIMARY KEY,
            Name TEXT,
            Department TEXT,
            Salary INTEGER,
            Hire_Date TEXT
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS Departments (
            ID INTEGER PRIMARY KEY,
            Name TEXT,
            Manager TEXT
        )
    """)

    cursor.executemany("INSERT OR IGNORE INTO Employees VALUES (?, ?, ?, ?, ?)", [
        (1, 'Alice', 'Sales', 50000, '2021-01-15'),
        (2, 'Bob', 'Engineering', 70000, '2020-06-10'),
        (3, 'Charlie', 'Marketing', 60000, '2022-03-20')
    ])

    cursor.executemany("INSERT OR IGNORE INTO Departments VALUES (?, ?, ?)", [
        (1, 'Sales', 'Alice'),
        (2, 'Engineering', 'Bob'),
        (3, 'Marketing', 'Charlie')
    ])

    conn.commit()
    conn.close()

initialize_db()

def execute_query(sql: str, params=()):
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute(sql, params)
    results = cursor.fetchall()
    conn.close()
    return results

@app.get("/query/")
def query_assistant(q: str = Query(..., description="Enter your question")):
    q = q.lower()

    if "employees in" in q:
        dept = q.split("employees in the ")[-1].replace(" department", "").strip()
        sql = "SELECT Name FROM Employees WHERE Department = ?"
        result = execute_query(sql, (dept,))
        return {"employees": [row[0] for row in result]} if result else {"message": "No employees found."}

    elif "manager of" in q:
        dept = q.split("manager of the ")[-1].replace(" department", "").strip()
        sql = "SELECT Manager FROM Departments WHERE Name = ?"
        result = execute_query(sql, (dept,))
        return {"manager": result[0][0]} if result else {"message": "Department not found."}

    elif "hired after" in q:
        date = q.split("hired after ")[-1].strip()
        sql = "SELECT Name FROM Employees WHERE Hire_Date > ?"
        result = execute_query(sql, (date,))
        return {"employees": [row[0] for row in result]} if result else {"message": "No employees found."}

    elif "total salary expense" in q:
        dept = q.split("total salary expense for the ")[-1].replace(" department", "").strip()
        sql = "SELECT SUM(Salary) FROM Employees WHERE Department = ?"
        result = execute_query(sql, (dept,))
        return {"total_salary": result[0][0]} if result[0][0] else {"message": "No salary data found."}

    else:
         return {"message": "Query not supported."}
